## Analysis 1 (merge/split clouds)

In [1]:
import datetime as dt
import itertools
from pathlib import Path

import dask
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from scipy import stats
from scipy import interpolate
import xarray as xr

from mcs_prime import McsTracks, PATHS
from mcs_prime.util import update_progress
%matplotlib inline

Load all tracks from 2000/6-2020/12

In [2]:
tracks = tracks = McsTracks.load(PATHS['statsdir'] / 'mcs_tracks_final_extc_20190101.0000_20200101.0000.nc', 
                                 PATHS['pixeldir'])
tracks

McsTracks[2019-01-01 00:30:00, 2019-12-31 22:30:00, ntracks=32843]

In [8]:
for trackid in tracks.dstracks.isel(tracks=tracks.dstracks.start_split_cloudnumber > 0).tracks.values[:1000]:
    t = tracks.get_track(trackid)
    sscn = t.dstrack.start_split_cloudnumber.values.item()
    found_prev_tracks = ((tracks.dstracks.cloudnumber == sscn) & (tracks.dstracks.base_time == t.base_time[0] - int(3600e9))).sum().values.item()
    # assert found_prev_tracks == 1
    print(trackid, ((tracks.dstracks.cloudnumber == sscn) & (tracks.dstracks.base_time == t.base_time[0] - int(3600e9))).sum().values.item())
    if found_prev_tracks != 1:
        break

39 1
40 1
41 0


In [13]:
(tracks.dstracks.cloudnumber == sscn).sum()

<xarray.DataArray 'cloudnumber' ()>
array(4320)

In [15]:
prev_tracks = tracks.dstracks.isel(tracks=(tracks.dstracks.base_time == t.base_time[0] - int(3600e9)).any(axis=1))

In [17]:
prev_tracks

<xarray.Dataset>
Dimensions:                  (tracks: 39, times: 400, mergers: 50, nmaxpf: 3)
Coordinates:
  * tracks                   (tracks) int64 0 1 2 3 4 5 6 ... 33 34 35 36 37 38
  * times                    (times) int64 0 1 2 3 4 5 ... 395 396 397 398 399
  * mergers                  (mergers) int64 0 1 2 3 4 5 6 ... 44 45 46 47 48 49
  * nmaxpf                   (nmaxpf) int64 0 1 2
Data variables: (12/55)
    track_duration           (tracks) int32 37 28 17 40 12 16 ... 10 19 12 15 21
    start_status             (tracks) float64 13.0 13.0 13.0 ... 2.0 1.0 1.0
    end_status               (tracks) float64 37.0 34.0 0.0 3.0 ... 3.0 21.0 3.0
    start_basetime           (tracks) datetime64[ns] 2019-01-01T00:30:00 ... ...
    end_basetime             (tracks) datetime64[ns] 2019-01-02T12:30:00 ... ...
    start_split_cloudnumber  (tracks) float64 nan nan nan nan ... nan nan nan
    ...                       ...
    pf_mcsstatus             (tracks, times) float64 1.0 1.0 1.0 ... nan nan nan
    movement_distance        (tracks, times) float32 28.28 14.14 ... nan nan
    movement_speed           (tracks, times) float32 7.857 3.928 ... nan nan
    movement_theta           (tracks, times) float32 -45.0 -45.0 ... nan nan
    movement_distance_x      (tracks, times) float32 20.0 10.0 10.0 ... nan nan
    movement_distance_y      (tracks, times) float32 -20.0 -10.0 ... nan nan
Attributes: (12/23)
    Title:                       Statistics of each MCS track
    Institution:                 Pacific Northwest National Laboratory
    Contact:                     Zhe Feng, zhe.feng@pnnl.gov
    Created_on:                  Mon Feb 28 05:50:54 2022
    startdate:                   20190101.0000
    enddate:                     20200101.0000
    ...                          ...
    PF_PF_min_majoraxis_thresh:  20
    coefs_pf_area:               [2874.05    89.825]
    coefs_pf_rr:                 [3.01657   0.0144461]
    coefs_pf_skew:               [0.194462  0.0100072]
    coefs_pf_heavyratio:         [3.419024 0.438709]
    max_speed_thresh:            50

In [25]:
np.nansum(t.dstrack.merge_ccs_area.values[:35, :], axis=1)

array([     0.,      0.,   2000.,      0.,      0.,      0.,      0.,
            0.,   5300.,  12300.,  64900., 105800., 120100.,  98600.,
         1700., 159300., 173700., 114900.,  12700.,  11000.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
         7100.,  21200.,  26200.,  42900.,  34700.,   2400.,      0.],
      dtype=float32)

In [27]:
np.nansum(t.dstrack.split_ccs_area.values[:35, :], axis=1)

array([     0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0., 167500., 176700.,  97800.,   1500.,      0.,   4400.,
         2300.,      0.,   1400.,      0.,   4500.,      0.,      0.,
         7700.,   6300.,      0.,      0.,      0.,      0.,      0.],
      dtype=float32)

In [29]:
t.dstrack.ccs_area.values[:35]

array([ 11300.,  17700.,  24300.,  27700.,  26600.,  23200.,  23400.,
        24300.,  24700.,  39800.,  42900.,  56400., 157800., 261000.,
       409400., 242900., 246100., 219800., 329000., 213300., 210600.,
       195100., 168200.,  95600., 107300.,  87800.,  95300.,  75100.,
        42700.,  17700.,  16300.,  22900.,  50000., 106400., 101800.],
      dtype=float32)